<h1 align=center><font size = 5>Capstone - ACME Startup Relocation in New York City</font></h1>

## Introduction/Problem

ACME Start is a Machine Learning (ML) startup in New York City that recently received Series A Funding in the amount of $20M USD. ACME currently has 10 employees and is looking to grow to 20 employees by years end.  The startup has outgrown its current location and needs to relocate to accommodate current and future employees.  

## Data
The Chief Executive Officer (CEO) is concerned that the relocation may result in staff attrition so the CEO asks the Human Resources (HR) Director to perform a survey of employees. The survey asks employees to rank factors that the company will incorporate in the selection of the new location. These factors include proximity to mass resturants, parks, and gyms. The HR Director will use these results of the survey and Foursquare location data to prepare a report of suitable locations for the relocation.


## Methodology
<img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DS0103EN/labs/images/lab1_fig2_datascience_methodology_flowchart.png" width = 500> 

### Business Understanding
The HR Director is the lead for this analysis.  She understands that office relocations can be stressful for employees and at time bring about unwanted attrition so she will prioritize employee engagement.

### Analytics Approach
The HR Director has asked the Data Science team to conduct a survey to of employees to gauge employee sentiment and use the information learned in that survey to guide the data analysis.

### Data Requirements
The Data Science team will center their analysis on three data sources:
- Employee Survey
- New York City Maps
- Foursquare Location Data

### Data Collection
The Data Science team has collected the raw survey results from employees using SurveyMonkey.  The team downloaded mapping files from the NY city website.  The team signed up for a Foursquare account to access data using the Foursquare API.

### Data Understanding
The Data Science team will use standard data exploration techniques on each data source to learn the structure and dimensions of all data.

### Data Preparation
The Data Science team will transform data, making sure their are no missing data values, and incorrect data types.  The team will impute missing data from available data. Also, the team will map Foursquare Venue Categories to the ranked features that employees called for in the survey. All data transformation steps will be documented in the Notebook.  

### Modeling
The Data Science team will furnish maps of potential sites and segment the sites by their features. The team decided to use clustering and segementation techniques as they perform well on geographical data where unsupervised learning is needed. The team will use K-means with K=5.

### Evaluation
The Data Science team will evaluate sites for suitability by determining what venues are within 200 meters of potential locations.  

### Deployment
The Data Science team will present findings to the HR Director and CEO.  

### Feedback
The Data Science team will solicit feedback from the HR Director and CEO, both of whom may ask for additional factors to be included in the analysis.





## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Datasets</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print('Matplotlib version: ', mpl.__version__) # >= 2.0.0

import re 

print('Libraries imported.')

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 
  - https://repo.continuum.io/pkgs/main/osx-64/_ipyw_jlab_nb_ext_conf-0.1.0-py36h2fc01ae_0.tar.bz2::_ipyw_jlab_nb_ext_conf-0.1.0-py36h2fc01ae_0, https://repo.continuum.io/pkgs/main/osx-64/conda-build-3.0.27-py36hb78d8cd_0.tar.bz2::conda-build-3.0.27-py36hb78d8cd_0, https://repo.continuum.io/pkgs/main/osx-64/conda-verify-2.0.0-py36he837df3_0.tar.bz2::conda-verify-2.0.0-py36he837df3_0, https://repo.continuum.io/pkgs/main/osx-64/navigator-updater-0.1.0-py36h7aee5fb_0.tar.bz2::navigator-updater-0.1.0-py36h7aee5fb_0
  - defaults::_ipyw_jlab_nb_ext_conf-0.1.0-py36h2fc01ae_0, https://repo.continuum.io/pkgs/main/osx-64/conda-build-3.0.27-py36hb78d8cd_0.tar.bz2::conda-build-3.0.27-py36hb78d8cd_0, https://repo.continuum.io/pkgs/main/osx-64/conda-verify-2.0

<a id='item1'></a>

## 1. Download and Explore Datasets

### Survey Dataset

The Chief Executive Officer (CEO) is concerned that the relocation may result in staff attrition so the CEO asks the Human Resources (HR) Director to perform a survey of employees. The survey asks employees to rank factors that the company will incorporate in the selection of the new location. Some of these factors include proximity to mass transit, childcare, parks, and gyms. The HR Director will use these results of the survey and Foursquare location data to prepare a report of suitable locations for the relocation.

In [2]:
raw_survey_df = pd.read_csv('ACME_Startup_Survey.csv')

FileNotFoundError: File b'ACME_Startup_Survey.csv' does not exist

In [ ]:
raw_survey_df.shape

In [ ]:
raw_survey_df

Fix data issue with empty data not identified with NaN

In [ ]:
raw_survey_df.replace(" ", np.nan, inplace = True)

Check the datatypes

In [ ]:
raw_survey_df.dtypes

Change the data type of Gyms to float64

In [ ]:
raw_survey_df['Gyms'] = raw_survey_df['Gyms'].astype("float")

Deal with missing data

In [ ]:
avgResturants = raw_survey_df['Resturants'].astype("float").mean(axis=0)
raw_survey_df["Resturants"].replace(np.nan, avgResturants, inplace=True)
print('Set missing Resturants to Average Resturants:',avgResturants)

avgGyms = raw_survey_df['Gyms'].astype("float").mean(axis=0)
raw_survey_df["Gyms"].replace(np.nan, avgGyms, inplace=True)
print('Set missing Gyms to Average Gyms:',avgGyms)

avgParks = raw_survey_df['Parks'].astype("float").mean(axis=0)
raw_survey_df["Parks"].replace(np.nan, avgParks, inplace=True)
print('Set missing Parks to Average Parks:',avgParks)

Saved transformed data as survey dataframe

In [ ]:
survey_df = raw_survey_df
survey_df

Perform 'Exploratory Data Analysis'

In [ ]:
survey_df.describe()

Visualize Data

In [ ]:
survey_df[['Resturants','Gyms','Parks']].plot(kind='box', figsize=(10, 7))

plt.title('Box plots of Survey Results')
plt.xlabel('Survey Responses')
plt.show()

Calculate the means to determine the ranking of factors that staff want to inform the decision

In [ ]:
factors_df=survey_df[['Resturants','Gyms','Parks']].mean(axis=0).reset_index()

In [ ]:
factors_df

In [ ]:
factors_df.columns=['Factor','Average']

Rank the factors

In [ ]:
factors_df['Rank']=factors_df.index + 1

Show the rank order of factors in order of importance to the employees

In [ ]:
factors_df

### Neighborhood Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.

In [ ]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

#### Load and explore the data

Next, let's load the data.

In [ ]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [ ]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [ ]:
neighborhoods_data[0]

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [ ]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [ ]:
neighborhoods

Then let's loop through the data and fill the dataframe one row at a time.

In [ ]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [ ]:
neighborhoods.head()

And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

#### Use geopy library to get the latitude and longitude values of Manhattan.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

#### Create a map of Manhattan with neighborhoods superimposed on top.

The company has chosen to relocate into a neighborhood in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [ ]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

Let's get the geographical coordinates of Manhattan.

In [ ]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [ ]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'FD0EOCRMLZBUH0QSGNF0NSNXMPAMT2I5OLGUSGQSAVMDCDZH' # your Foursquare ID
CLIENT_SECRET = 'QXC0KBN1WHQ0I40KKACAPQRC2OS1EBKOWCE0LWBSUJD1IDMS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [ ]:
manhattan_data.loc[0, 'Neighborhood']

Get the neighborhood's latitude and longitude values.

In [ ]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

#### Now, let's get the top 20 venues that are in Marble Hill within a radius of 200 meters (roughly 1 city block). By focusing on items nearest to the relocation center we will demonstrate to employees we valued their survey responses.

First, let's create the GET request URL. Name your URL **url**.

In [ ]:
# type your answer here

LIMIT = 20 # limit of number of venues returned by Foursquare API

radius = 200 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL




Send the GET request and examine the resutls

In [ ]:
results = requests.get(url).json()
results

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

And how many venues were returned by Foursquare?

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

<a id='item2'></a>

## 2. Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [ ]:
# type your answer here

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [ ]:
print(manhattan_venues.shape)
manhattan_venues.head()

Let's explore the Venue Categories

In [ ]:
#manhattan_venues[['Venue Category']].groupby(['Venue Category']).size().reset_index()

We see that we need to map 'Venue Category' to 'Factors' that the employees ranked for us.  

In [ ]:
factors_df

Let's map all resturants and bars to 
manhattan_venues[['Venue Category']].groupby(['Venue Category']).size()

In [ ]:
#df[df.narr1.str.contains("F[AE]LL.*BI[KC]")]
#manhattan_venues['Venue Category'].str.contains("Restaurant")

In [ ]:
# Importing re package for using regular expressions 
import re 

In [ ]:
# Function to map venue categories to employee factors 
def Map_Venue_Category(Venue_Category): 
    
    #print (Venue_Category)
    if re.search('Restaurant', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant' 
    
    if re.search('Joint', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant' 
    
    if re.search('Tea Room', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant' 
    
    if re.search('Pizza Place', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant' 
    
    if re.search('Donut Shop', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant' 
    
    if re.search('Coffee Shop', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Bagel Shop', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Bakery', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Bar', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Bistro', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Beer', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Burrito', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Breakfast', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Tea Shop', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Candy', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Café', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Cheese', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Chocolate', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Bridge', Venue_Category, re.IGNORECASE): 
        
        return 'MassTransit'
    
    if re.search('Bus Line', Venue_Category, re.IGNORECASE): 
        
        return 'MassTransit'
    
    if re.search('Community Center', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    if re.search('Cupcake', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Deli', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Dessert', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Diner', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Dog Run', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Farmers Market', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Food', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Fountain', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    if re.search('Yogurt', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Gourmet', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    if re.search('Grocery', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Gym', Venue_Category, re.IGNORECASE): 
        
        return 'Gym'
    
    if re.search('Harbor', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    if re.search('Histor', Venue_Category, re.IGNORECASE): #for historic and history
        
        return 'Park'
    
    if re.search('ice cream', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Liquor', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Memorial', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    if re.search('Pie', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Poke Place', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Pool', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    if re.search('Snack', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Steak', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Taco', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Tennis Stadium', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    if re.search('Trail', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    if re.search('Field', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    if re.search('Spa', Venue_Category, re.IGNORECASE): 
        
        return 'Gym'
    
    if re.search('Golf Course', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Playground', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Plaza', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    if re.search('Basketball Court', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    if re.search('GastroPub', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Momument', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    if re.search('Sandwich', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Gastro', Venue_Category, re.IGNORECASE): 
        
        return 'Restaurant'
    
    if re.search('Tennis', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    if re.search('Waterfront', Venue_Category, re.IGNORECASE): 
        
        return 'Park'
    
    else: 
        # if clean up needed return the same name 
        return Venue_Category 
          
# Set the Employee Venue Category
manhattan_venues['Employee_Venue_Category'] = manhattan_venues['Venue Category'].apply(Map_Venue_Category)
  
# Print the updated dataframe 
manhattan_venues[['Employee_Venue_Category']].groupby(['Employee_Venue_Category']).size()

Let's check how the venues were returned for each neighborhood

In [ ]:
manhattan_venues[['Neighborhood','Employee_Venue_Category']].groupby(['Neighborhood','Employee_Venue_Category']).size()

#### Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} unique employee venue categories.'.format(len(manhattan_venues['Employee_Venue_Category'].unique())))

<a id='item3'></a>

## 3. Analyze Each Neighborhood by Employee Mapped Venue Categories

In [ ]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Employee_Venue_Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

And let's examine the new dataframe size.

In [ ]:
manhattan_onehot.shape

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

#### Let's confirm the new size

In [ ]:
manhattan_grouped.shape

#### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [ ]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Cluster 2

In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Cluster 3

In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Cluster 4

In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

#### Cluster 5

In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

# Results

Find the location that has all of the venues in the rank order chosen by the employees



In [ ]:
manhattan_merged.loc[(manhattan_merged['1st Most Common Venue'] == 'Restaurant') & 
                     (manhattan_merged['2nd Most Common Venue'] == 'Gym') & 
                     (manhattan_merged['3rd Most Common Venue'] == 'Park')]

## Reccommendation based on results - Carnegie Hill 
Based on the analysis we found that the Neighborhood of Carnegie Hill has all the venues in rank order that the employees of the startup selected in their survey.

# Discussion

The key element of this analysis was clearly the ranking of factors by employees. This was by design as the HR Director wanted to ensure employees felt they were heard and respected in this office relocations.  As a startup it is critical to maintain key staff members.  The analysis that the Data Science team undertook found the one location in Manhattan, NY that met all criteria.

Potentially, the analysis could be tweaked by adding new survey questions for employees or weighting the response of developers and data scientist over others in the company. Also, the search radius around each location could be increased from 200 meters (1 city block) to 500 meters(2.5 city blocks) to get more information on nearby venues.  However, that could lead to noise and potentially overlapping locations.

# Conclusion

The analysis met its desired outcome by identifying the single location in Manhattan that clear matched all the ranked desires of the staff.